In [ ]:
# !pip install faiss-cpu
# !pip install gensim
# !pip install pybind11
# !pip install numpy<2


In [ ]:
import faiss # chaining vectors into one meaningful one
from gensim.models import Word2Vec
import numpy as np

import nltk # to convert text into some more presentable way
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def sentenses_convert(file_name):
  with open(file_name, "r", encoding="utf-8") as file: ## reading a file and storing it into one variable
    f = file.read()

  sentenses = sent_tokenize(f)
  tokenized_sentenses = [word_tokenize(sent) for sent in sentenses] # structure a text

  model = Word2Vec(tokenized_sentenses, vector_size=100, window=5, min_count=2, sg=1) # convertation to vectors
  document_vectors = [] # TODO
  for sentence in tokenized_sentenses:
      sentence_vector = np.mean([model.wv[word] for word in sentence if word in model.wv], axis=0)
      if np.isnan(sentence_vector).any():
          sentence_vector = np.zeros(model.vector_size)
      document_vectors.append(sentence_vector)
  return document_vectors, model


In [ ]:
doc_vec, model = sentenses_convert("church_text")

In [ ]:
def faiss_convertion(document, model): # now we collect our vectors into matrices

  document_vectors = np.array(document).astype("float32")

  index = faiss.IndexFlatL2(model.vector_size)
  index.add(document_vectors)  # add vectors to FAISS
  return index, model

In [ ]:
faiss_index, faiss_model = faiss_convertion(doc_vec,model)

In [ ]:
### models part
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.model_selection import train_test_split

In [ ]:
phi = "microsoft/phi-2"
phi_model = AutoModelForCausalLM.from_pretrained(
    phi,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    trust_remote_code=True ## fix api issues
    )
phi_tokenizer = AutoTokenizer.from_pretrained(phi)